In [1]:
import pandas as pd
import pickle
import re
from preprocessing import process
import swifter

Reading the data

In [3]:
data = pd.read_csv('~/Downloads/one-million-reddit-jokes.csv')
data

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,ftbp1i,2qh72,jokes,False,1585785543,https://old.reddit.com/r/Jokes/comments/ftbp1i...,self.jokes,NaN,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,2
1,post,ftboup,2qh72,jokes,False,1585785522,https://old.reddit.com/r/Jokes/comments/ftboup...,self.jokes,NaN,It's called Google Sheets.,Did you know Google now has a platform for rec...,9
2,post,ftbopj,2qh72,jokes,False,1585785508,https://old.reddit.com/r/Jokes/comments/ftbopj...,self.jokes,NaN,The vacuum doesn't snore after sex.\n\n&amp;#x...,What is the difference between my wife and my ...,15
3,post,ftbnxh,2qh72,jokes,False,1585785428,https://old.reddit.com/r/Jokes/comments/ftbnxh...,self.jokes,NaN,[removed],My last joke for now.,9
4,post,ftbjpg,2qh72,jokes,False,1585785009,https://old.reddit.com/r/Jokes/comments/ftbjpg...,self.jokes,NaN,[removed],The Nintendo 64 turns 18 this week...,134
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,post,30ekhl,2qh72,jokes,False,1427396885,https://old.reddit.com/r/Jokes/comments/30ekhl...,self.jokes,NaN,*zyan malik or whatever leaves 1d. \n*Kayne W...,With Zyan Malik leaving 1D..,0
999996,post,30ekft,2qh72,jokes,False,1427396863,https://old.reddit.com/r/Jokes/comments/30ekft...,self.jokes,NaN,[deleted],Why did the exterminator go to Italy?,1
999997,post,30ek9i,2qh72,jokes,False,1427396794,https://old.reddit.com/r/Jokes/comments/30ek9i...,self.jokes,NaN,I'll be Bach,What did Arnold Schwarzenegger say when invite...,0
999998,post,30ek3q,2qh72,jokes,False,1427396721,https://old.reddit.com/r/Jokes/comments/30ek3q...,self.jokes,NaN,So a moth goes into a podiatrists office.\n\n\...,The Moth Joke,87


Remove the columns that we don't need

In [4]:
data = data[['selftext', 'title', 'score']]

Check if some column contains Nan values

In [5]:
data.isna().sum()

selftext    4475
title          0
score          0
dtype: int64

Remove Nan values

In [6]:
data.dropna(inplace=True)

/var/folders/2w/9zd5xr8x3qg27qldfljyr06r0000gn/T/ipykernel_84779/1368182302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [7]:
data.isna().sum()

selftext    0
title       0
score       0
dtype: int64

In [24]:
data.head(10)

,selftext,title,score
0,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,2
1,It's called Google Sheets.,Did you know Google now has a platform for rec...,9
2,The vacuum doesn't snore after sex.\n\n&amp;#x...,What is the difference between my wife and my ...,15
3,[removed],My last joke for now.,9
4,[removed],The Nintendo 64 turns 18 this week...,134
5,[removed],Sex with teacher.,1
6,[removed],Another long one.,8
7,Oo..lala...,What did the French man say to the attractive ...,2
8,[removed],A Priest takes a walk down to the docks one day,88
9,[removed],A long joke.,4


I would like to get rid of values like [removed], let's do it using regular expressions

In [8]:
data = data[data.selftext.apply(lambda text: re.fullmatch(r'\[(\w)+\]', text) is None)]

Let's sort rows by the score. The higher score  document has, the more relevant it is.

In [9]:
data = data.sort_values('score', ascending=False, ignore_index=True)

In [33]:
data

,selftext,title,score
0,"RIP Larry Tesler, the UI designer that created...",Sad News: The founder of /r/jokes has passed away,142733
1,Paying additional money to access certain site...,Calm down about the Net Neutrality thing...,136359
2,V\n\n*Edit: seems like the ctrl key on my keyb...,V,106412
3,Imagine how surprised he must have been.\n\nEd...,If your surprised that Jeffrey Epstein commite...,103652
4,"He speaks with the officer, who assigns him hi...",A new Navy recruit has his first day on the su...,98257
...,...,...,...
573986,"He said ""You are what you eat!""",I called Jeffrey Dahmer a faggot,0
573987,The Judicial Branch,What's hitler's least favorite branch of gover...,0
573988,"There were too many sticks in my front yard, s...",There's a faggot in the trunk of my car.,0
573989,When she though it snowed.\n\n-No it is reindeer.,What did Santa Claus say to Mrs. Claus.,0


Let's process the titles and the texts of these jokes. Here we get rid of punctuation marks, remove the stop words, lemmatize the words in the titles and the texts. Should the stop words de deleted? That's a good question.

In [12]:
data['prep_title'] = data.title.swifter.apply(process)

Pandas Apply:   0%|          | 0/573991 [00:00<?, ?it/s]

In [13]:
data['prep_text'] = data.selftext.swifter.apply(process)

Pandas Apply:   0%|          | 0/573991 [00:00<?, ?it/s]

In [14]:
data

,selftext,title,score,prep_title,prep_text
0,"RIP Larry Tesler, the UI designer that created...",Sad News: The founder of /r/jokes has passed away,142733,sad news founder r joke pass away,rip larry tesler ui designer create cut copy p...
1,Paying additional money to access certain site...,Calm down about the Net Neutrality thing...,136359,calm net neutrality thing,pay additional money access certain site give ...
2,V\n\n*Edit: seems like the ctrl key on my keyb...,V,106412,v,v edit seem like ctrl key keyboard work
3,Imagine how surprised he must have been.\n\nEd...,If your surprised that Jeffrey Epstein commite...,103652,surprised jeffrey epstein commit suicide morning,imagine surprise must edit thank redditors awa...
4,"He speaks with the officer, who assigns him hi...",A new Navy recruit has his first day on the su...,98257,new navy recruit first day submarine,speaks officer assign post go stand periscope ...
...,...,...,...,...,...
573986,"He said ""You are what you eat!""",I called Jeffrey Dahmer a faggot,0,call jeffrey dahmer faggot,say eat
573987,The Judicial Branch,What's hitler's least favorite branch of gover...,0,hitler least favorite branch government,judicial branch
573988,"There were too many sticks in my front yard, s...",There's a faggot in the trunk of my car.,0,faggot trunk car,many stick front yard dump wood
573989,When she though it snowed.\n\n-No it is reindeer.,What did Santa Claus say to Mrs. Claus.,0,santa claus say mr claus,though snow reindeer


In [105]:
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

Creating inverted index

In [107]:
def build_inverted_index(data):
    inv_index = {}

    for idx, row in enumerate(data.values):
        for word in set(row.split()):
            if word not in inv_index:
                inv_index[word] = []
            inv_index[word].append(idx)

    return inv_index

In [108]:
title_inv_index = build_inverted_index(data.prep_title)
text_inv_index = build_inverted_index(data.prep_selftext)

In [110]:
with open('title_inv_index.pickle', 'wb') as f:
    pickle.dump(title_inv_index, f)

In [111]:
with open('text_inv_index.pickle', 'wb') as f:
    pickle.dump(text_inv_index, f)